In [1]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
from vincenty import vincenty
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import geoplotlib as gp
import scipy as sp
import gdal 
from gdalconst import * 
import datetime
import math
import operator
import re
import os
import os.path
import requests, time
import urllib, json
import csv
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
tls.set_credentials_file(username='ShawnHou', api_key='OohmqCJaPfb6p6xo4pDF')
from geoplotlib.utils import BoundingBox
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import euclidean_distances
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from pprint import pprint
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from scipy.spatial import distance
from collections import defaultdict
from operator import itemgetter
from pushover import init, Client
from scipy.spatial import KDTree
from calendar import monthrange
from sklearn import metrics
import tarfile

#for parallel processing
from ipyparallel import Client

%matplotlib inline

C:\WorkingGround\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [ ]:
years_stat=dict()
for YEAR in range(2017,1979,-1):
    year_file=os.getcwd()+'\\data\\data option one\\'+str(YEAR)+'.csv.gz'
    year_df = pd.read_csv(year_file, compression='gzip', header=None,usecols = used_columns,
                              dtype=dict(zip(used_columns, dtypes)),sep=',').rename(columns=dict(zip(used_columns, data_columns)))
    #year_station=year_df[ year_df['GHCND'].isin(temp_stations_clean['GHCND'].tolist())
                                 #   & ( year_df['ATTRIBUTE'].isin([metric])) & ( year_df['VALUE']!=-999) ]
    year_station_df=pd.merge(year_df,temp_stations_clean, how='right', on="GHCND")
    grouped_year_station= year_station_df.groupby(['ATTRIBUTE'])
    for name, group in grouped_year_station:
        group_sup_df=group[['GHCND','DATE','VALUE','LAT','LON','ELEV','ST']]
        if name in wanted_metrics:
            group_sup_df.to_csv('data\\daily GHCND dataframes\\{}.csv'.format(str(YEAR)+"_"+name),header=False, index_label=False)
    #years_stat[YEAR]=year_station_df
    print 'Year: ',YEAR, ' - ', year_station_df.info()
    print "***********************************************************************"
    

In [2]:
temp_stations = pd.read_csv(os.getcwd()+'\\data\\data option one\\weather_station_list.csv',skipinitialspace=True)
temp_stations['GHCND']=temp_stations.GHCND.str.strip()
temp_stations=temp_stations[temp_stations.GHCND.str.contains("US")] ##Discard non US stations
temp_stations_clean=temp_stations[np.abs(temp_stations.LON-temp_stations.LON.mean())<=(5*temp_stations.LON.std())] ##discard OUTLIER by location

print "include outliner, size:",temp_stations.shape
print "exclude outliner, size:",temp_stations_clean.shape

include outliner, size: (56918, 6)
exclude outliner, size: (56911, 6)


#### Avoid reexecuting following section !!!! Due to connection to remote web service

In [ ]:
year="2016"
url_base = "http://data.fcc.gov/api/block/find?"
county_fips=list()
county_names=list()
block_fips=list()
print temp_stations_clean.shape
i=0
for index, row in temp_stations_clean.iterrows():
    lat=str(row['LAT'])
    lon=str(row['LON'])
    url_geo="latitude="+lat+"&"+"longitude="+lon+"&format=json"
    url=url_base+url_geo
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    county_fips.append(data['County']['FIPS'])
    county_names.append(data['County']['name'])
    block_fips.append(data['Block']['FIPS'])
    i=i+1
    if i%1000==0:
        print i
    if i%15000==0:
        time.sleep(600)
        

county_fips_s = pd.Series(county_fips)
county_name_s = pd.Series(county_names)
block_fips_s = pd.Series(block_fips)
temp_stations_clean['COUNTY_CODE'] = county_fips_s.values
temp_stations_clean['COUNTY_NAME'] = county_name_s.values
temp_stations_clean['BLOCK_CODE'] = block_fips_s.values

In [ ]:
temp_stations_clean.to_csv('data\\daily dataframe\\cleaned_weather_station_list_withCounty.csv', index=False,encoding="UTF-8")